In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from google.colab import files
import pandas as pd
import re
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset


In [ ]:
!pip install datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import joblib

# Дополнительные импорты для улучшенной очистки
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

from datasets import Dataset, DatasetDict


In [ ]:
import os
import pandas as pd
import re
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset, ClassLabel, Features, Value
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import joblib


In [1]:
from google.colab import files

# Загрузка файла
uploaded = files.upload()
file_name = next(iter(uploaded))
df = pd.read_excel(file_name, sheet_name='ГОТОВО 82т 27sep')


Saving ТХС 14разд_12Jul24.xlsx to ТХС 14разд_12Jul24.xlsx


NameError: name 'pd' is not defined

In [ ]:
# Загрузка необходимых ресурсов NLTK
nltk.download('stopwords')
nltk.download('wordnet')

# Инициализация стоп-слов и лемматизатора
stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Константы
SAVE_DIR = './model_output'
MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_EPOCHS = 3
LEARNING_RATE = 2e-5

# Настройка NLTK
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r'[\d.,*/\\]', '', text)
    text = re.sub(r'\b(грамм|г|кг|л|мл|шт|вес)\b', '', text)
    text = text.lower()
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens).strip()

def load_and_prepare_data():
    file_path = '/content/Готово 82т_27sep24.xlsx'
    print(f"Загрузка данных из файла: {file_path}")

    try:
        df = pd.read_excel(file_path)
        print("Файл успешно загружен.")
    except Exception as e:
        print(f"Ошибка при загрузке файла: {e}")
        raise

    print("Названия столбцов в загруженном файле:", df.columns.tolist())

    # Предполагаем, что первый столбец - названия продуктов, третий - категории
    df['product_name'] = df.iloc[:, 0].apply(clean_text)
    df['category'] = df.iloc[:, 2].apply(clean_text)
    df = df[['product_name', 'category']].dropna()

    label_encoder = LabelEncoder()
    df['label'] = label_encoder.fit_transform(df['category'])

    class_counts = df['label'].value_counts()
    print("Распределение классов до удаления редких классов:")
    print(class_counts)

    # Удаление классов с менее чем 2 экземплярами
    df = df[df['label'].isin(class_counts[class_counts >= 2].index)]
    df['label'] = label_encoder.fit_transform(df['category'])

    print("Распределение классов после удаления редких классов:")
    print(df['label'].value_counts())

    # Создаем Dataset напрямую из словаря
    dataset_dict = {
        'product_name': df['product_name'].tolist(),
        'label': df['label'].tolist()
    }

    features = Features({
        'product_name': Value('string'),
        'label': ClassLabel(names=label_encoder.classes_.tolist())
    })

    dataset = Dataset.from_dict(dataset_dict, features=features)
    return dataset.train_test_split(test_size=0.2, seed=42, stratify_by_column='label'), label_encoder

def prepare_model_and_train(train_dataset, test_dataset, num_labels):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

    def tokenize_function(examples):
        return tokenizer(examples['product_name'], truncation=True, padding='max_length', max_length=MAX_LENGTH)

    train_dataset = train_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        weight_decay=0.01,
        logging_dir=os.path.join(SAVE_DIR, 'logs'),
        logging_steps=10,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
    )

    def compute_metrics(p):
        preds = p.predictions.argmax(-1)
        precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
        acc = accuracy_score(p.label_ids, preds)
        return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    results = trainer.evaluate()
    print("Результаты оценки модели:", results)

    return model, tokenizer

if __name__ == "__main__":
    dataset, label_encoder = load_and_prepare_data()
    train_dataset, test_dataset = dataset['train'], dataset['test']

    print(f"Размер обучающей выборки: {len(train_dataset)}")
    print(f"Размер тестовой выборки: {len(test_dataset)}")

    model, tokenizer = prepare_model_and_train(train_dataset, test_dataset, len(label_encoder.classes_))

    os.makedirs(SAVE_DIR, exist_ok=True)
    model.save_pretrained(SAVE_DIR)
    tokenizer.save_pretrained(SAVE_DIR)
    joblib.dump(label_encoder, os.path.join(SAVE_DIR, 'label_encoder.pkl'))

    print(f"Модель, токенизатор и LabelEncoder успешно сохранены в директории '{SAVE_DIR}'.")

Загрузка данных из файла: /content/Готово 82т_27sep24.xlsx


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Файл успешно загружен.
Названия столбцов в загруженном файле: ['Ягода Главпродукт Абрикос протертый с сахаром 550 гр 1/12', 'Абрикос с сахаром', '9.1.1 ФРУКТЫ КОСТОЧКОВЫЕ']
Распределение классов до удаления редких классов:
label
111    27754
162     4868
155     2627
119     2251
107     1836
       ...  
28         1
78         1
29         1
1          1
54         1
Name: count, Length: 176, dtype: int64
Распределение классов после удаления редких классов:
label
103    27754
154     4868
147     2627
111     2251
99      1836
       ...  
117        3
22         3
92         2
23         2
128        2
Name: count, Length: 168, dtype: int64
Размер обучающей выборки: 65883
Размер тестовой выборки: 16471


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.843800,0.850195,0.814826,0.782169,0.777664,0.814826
2,0.530400,0.526477,0.878878,0.865373,0.865799,0.878878
3,0.327100,0.450370,0.895210,0.884003,0.881603,0.895210


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Результаты оценки модели: {'eval_loss': 0.4503698945045471, 'eval_accuracy': 0.8952097626130775, 'eval_f1': 0.8840027482666297, 'eval_precision': 0.8816025286880597, 'eval_recall': 0.8952097626130775, 'eval_runtime': 62.7061, 'eval_samples_per_second': 262.67, 'eval_steps_per_second': 16.426, 'epoch': 3.0}
Модель, токенизатор и LabelEncoder успешно сохранены в директории './model_output'.


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import joblib

# Путь к сохраненной модели
MODEL_DIR = './model_output'

# Загрузка модели, токенизатора и label_encoder
model = DistilBertForSequenceClassification.from_pretrained(MODEL_DIR)
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_DIR)
label_encoder = joblib.load(f"{MODEL_DIR}/label_encoder.pkl")

# Функция для предсказания категории
def predict_category(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    predicted_category = label_encoder.inverse_transform([predicted_class])[0]
    confidence = probabilities[0][predicted_class].item()
    return predicted_category, confidence

# Интерактивное тестирование
print("Введите название продукта для классификации (или 'выход' для завершения):")
while True:
    user_input = input("> ")
    if user_input.lower() == 'выход':
        break
    category, confidence = predict_category(user_input)
    print(f"Предсказанная категория: {category}")
    print(f"Уверенность: {confidence:.2%}")
    print()

print("Тестирование завершено.")

Введите название продукта для классификации (или 'выход' для завершения):
> доширак
Предсказанная категория: фастфуд
Уверенность: 99.42%

> спагетти
Предсказанная категория: изделия зерна твердой пшеницы
Уверенность: 86.55%

> Пастила
Предсказанная категория: кулинарные изделия макаронами
Уверенность: 26.44%

> Зефир, глазированный шоколадом 
Предсказанная категория: пастила зефир
Уверенность: 81.75%

> Шоколад горький 
Предсказанная категория: шоколад добавления шоколад какао-порошок
Уверенность: 33.94%

> Ядро ореха фундука
Предсказанная категория: орехи
Уверенность: 16.76%

> Шикозин
Предсказанная категория: несъедобное
Уверенность: 81.01%

> Сироп сладкий
Предсказанная категория: кондитерское сырьё
Уверенность: 61.14%

>  Петушок леденец карамель с
Предсказанная категория: карамель
Уверенность: 85.65%

> СЫР МАСКАРПОНЕ
Предсказанная категория: сыры сычужные твердые
Уверенность: 46.06%

> СЫР МАСКАРПОНЕ GALBANI
Предсказанная категория: сыры сычужные твердые
Уверенность: 43.95%

> СЫ